In [9]:
import os
import numpy as np
import gdown
from pathlib import Path

import tensorflow as tf
from tensorflow import keras

In [10]:
url_path_map = {
        'train': {
            'url':'https://drive.google.com/uc?id=1JewlmOsqs-O1EmMMKutcYYiD7GkN6brN', 
            'path':'dataset/train.npy'
            },
        'test': {
            'url':'https://drive.google.com/uc?id=1JcCFZbc_N7VIa3G3XA8grwz4XGdaaXSZ', 
            'path':'dataset/test.npy'
            },
        'label': {
            'url':'https://drive.google.com/uc?id=1JiaTOcZ6QfDThw3RrIa3ZxDDaNt-BMUQ',
            'path':'dataset/label_info.txt'
        }
    }

In [11]:
def download_dataset():
    Path('dataset').mkdir(exist_ok=True)
    for split in ['train', 'test', 'label']:
        if Path(url_path_map[split]['path']).exists():
            continue
        gdown.download(url_path_map[split]['url'], url_path_map[split]['path'], quiet=False)

In [12]:
download_dataset()

Permission denied: https://drive.google.com/uc?id=1JewlmOsqs-O1EmMMKutcYYiD7GkN6brN
Maybe you need to change permission over 'Anyone with the link'?
Permission denied: https://drive.google.com/uc?id=1JcCFZbc_N7VIa3G3XA8grwz4XGdaaXSZ
Maybe you need to change permission over 'Anyone with the link'?


In [5]:
def read_txt(file_name):
    with open(file_name, 'rt') as f:
        class_list = [row.strip('\n') for row in f.readlines()]
    return class_list

In [6]:
train_dir = '/content/dataset/train.npy'
test_dir = '/content/dataset/test.npy'
label_dir = '/content/dataset/label_info.txt'

In [7]:
train = np.load(train_dir, allow_pickle=True).item()
test = np.load(test_dir, allow_pickle=True).item()

FileNotFoundError: ignored

In [ ]:
train_image , train_label = train['image'], train['label']
test_image, test_label = test['image'], test['label']

In [ ]:
train_image.shape, train_label.shape

In [ ]:
test_image.shape, test_label.shape

In [ ]:
train_image = np.split(train_image, 10, axis=3)
test_image = np.split(test_image, 10, axis=3)

In [ ]:
train_image = np.array(train_image)
test_image = np.array(test_image)

In [ ]:
train_image.shape, test_image.shape

In [ ]:
train_image = train_image.reshape((-1, 224, 224, 3))
test_image = test_image.reshape((-1, 224, 224, 3))

In [ ]:
train_image.shape, test_image.shape

In [ ]:
label_temp = np.append(train_label, train_label) #2
label_temp = np.append(label_temp, label_temp) #4
label_temp = np.append(label_temp, label_temp) #8
label_temp = np.append(label_temp, train_label) #9
train_label_extended = np.append(label_temp, train_label) #10

In [ ]:
train_label_extended.shape

In [ ]:
import matplotlib.pyplot as plt

#plt.figure(figsize=(3, 10))
for i in range(30):
  plt.subplot(3, 10, i+1)
  plt.imshow(train_image[i])

In [ ]:
plt.imshow(train_image[54])

In [ ]:
label_list = read_txt(label_dir)

In [ ]:
label_list

In [ ]:
train_image = train_image.astype('float32') / 255.0
test_image = test_image.astype('float32') / 255.0

In [ ]:
from tensorflow.keras import models, layers

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D

In [ ]:
model = models.Sequential()

In [ ]:
model.add(layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=(224, 224, 3)))
model.add(layers.MaxPooling2D(pool_size=(2,2)))

model.add(layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2,2)))

model.add(layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu'))
model.add(layers.MaxPooling2D(pool_size=(2,2)))

model.add(layers.Flatten())
model.add(layers.Dense(units=50, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras import optimizers

In [ ]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [ ]:
model.fit(x=train_image, y=train_label_extended, epochs=30, batch_size = 128, validation_split = 0.1)

In [ ]:
loss, accuracy = model.evaluate(x=train_image, y=train_label_extended)

In [ ]:
predictions = model.predict(test_image)

In [ ]:
np.argmax(predictions[4999])

In [ ]:
import pandas as pd

In [ ]:
id=[]
pred=[]
for i in range(500):
  id.append(i)
  pred.append(np.argmax(predictions[i]))

In [ ]:
challenge = pd.DataFrame({'image_id':id, 'label':pred})

In [ ]:
challenge

In [ ]:
#challenge.to_csv('deepChallenge08.csv', index=False)